In [2]:
import pandas as pd
import re
import proselint
import spacy
import nltk
from nltk.corpus import stopwords
from collections import Counter

from nltk import pos_tag, word_tokenize
import copy
import swifter

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')

/home/vignesh-aravindh-b/Documents/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /home/vignesh-
[nltk_data]     aravindh-b/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vignesh-
[nltk_data]     aravindh-b/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/vignesh-aravindh-b/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/vignesh-
[nltk_data]     aravindh-b/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train_df = pd.read_csv("train.tsv", sep = '\t')
test_df = pd.read_csv("test.tsv", sep = '\t')

In [4]:
train_df.head()

,Unnamed: 0,title,text,subject,date,label
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017",1
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017",0
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017",1
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016",0
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0


In [5]:
new_train_df = copy.deepcopy(train_df)
new_train_df.head()

,Unnamed: 0,title,text,subject,date,label
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017",1
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017",0
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017",1
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016",0
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0


## Step 1 : Linguistic Features
### Punctuation Check
### Grammar Check
### Readability Check

In [6]:
# # Punctuation
# text1 = "Hello my name is Aditya I love coding!!!!!! but, sometimes I make mistakes"
# issues = proselint.tools.lint(text1)
# if(not len(issues)):
#     print("No Puntuation issues found")
# else:
#     print("Issues Found")
i = 0
def check_punctuation(headline):
    issues = proselint.tools.lint(headline)
    if(not len(issues)):
        return 1
    else:
        return 0

new_train_df['Punctuate'] = new_train_df['title'].swifter.apply(check_punctuation)

Pandas Apply: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [01:03<00:00, 474.73it/s]


In [10]:
# new_train_df['Punctuate'].head(20)

0     1
1     0
2     1
3     0
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
Name: Punctuate, dtype: int64

In [10]:
# Grammar

nlp = spacy.load("en_core_web_sm")

def grammar_check(text):
    doc = nlp(text)
    issues = []

    for token in doc:
        # Check if the subject-verb agreement seems off
        if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
            if token.tag_ in ["NN", "NNP"] and token.head.tag_ == "VBP":
                issues.append(f"Issue: Singular subject '{token.text}' with plural verb '{token.head.text}'")
            elif token.tag_ in ["NNS"] and token.head.tag_ == "VBZ":
                issues.append(f"Issue: Plural subject '{token.text}' with singular verb '{token.head.text}'")

        # Flag any auxiliary verbs that don't seem to match the context
        if token.dep_ == "aux" and token.head.pos_ == "VERB":
            if token.text.lower() in ["is", "was"] and token.head.tag_ == "VBP":
                issues.append(f"Issue: Incorrect auxiliary verb '{token.text}' for verb '{token.head.text}'")

    return issues

# Example text
text1 = "Breaking news world leader meets to !!!!!!discuss climate change."
grammar_issues = grammar_check(text1)

if grammar_issues:
    print("Grammar Issues Found:")
    for issue in grammar_issues:
        print(issue)
else:
    print("No grammar issues found.")


[('hyperbolic.misc', "'coding!!!!!!' is hyperbolic.", 0, 31, 31, 44, 13, 'warning', None), ('leonard.exclamation.30ppm', 'More than 30 ppm of exclamations. Keep them under control.', 0, 37, 37, 38, 1, 'warning', None), ('leonard.exclamation.multiple', 'Stop yelling. Keep your exclamation points under control.', 0, 38, 38, 43, 5, 'warning', None)]


TypeError: tuple indices must be integers or slices, not str

## Step 2 : Keyword Density

In [ ]:
def keyword_density(text, keyword):
    """Calculates keyword density.

    Args:
        text: The input text.
        keyword: The keyword to analyze.

    Returns:
        The keyword density as a percentage.
    """

    # Tokenize the text and remove stop words
    words = [word.lower() for word in nltk.word_tokenize(text) if word.lower() not in stopwords.words('english') and word.isalnum()]

    # Count the occurrences of the keyword
    keyword_count = words.count(keyword.lower())

    # Calculate the keyword density
    density = (keyword_count / len(words)) * 100 if words else 0

    return density

text2 = "BONKERS BERNIE SANDERS: Prioritizing Jobs Over Climate Change Is ‘Stupid and Dangerous’ [Video]	https://www.youtube.com/watch?v=GPqQIlWksbgVermont Sen. Bernard Sanders told CNN Thursday that President Trump s plan to prioritize job creation over climate change regulations is  stupid and dangerous.  The commander in chief signed an  Energy Independence  executive order Tuesday as a means of rolling back former President Barack Obama s Clean Power Plan. Mr. Trump said his policies will cause  wealth to pour into our communities,  although Mr. Sanders frames it as a recipe for environmental disaster.  [This plan is a] nonsensical, and stupid, and dangerous approach,  Mr. Sanders told CNN s Wolf Blitzer Thursday.  It s almost indescribable. Look, the scientific community is virtually unanimous. While Trump and his friends think climate change is a hoax, what the scientists are telling us it is real, it is caused by human activity. It is already causing devastating problems.  The Trump administration argues on the White House website that its actions will prevent $39 billion in electricity hikes, allow 242 million tons of coal to benefit the American economy, and disbands the Interagency Working Group on the Social Cost of Greenhouse Gases.  I am taking historic steps to lift restrictions on American energy, to reverse government intrusion and to cancel job-killing regulations,  Mr. Trump said Tuesday while surrounded by coal industry executives. Kentucky Coal Association President Tyler White cheered the move soon afterward.Read more: WT	politics."
sensational_words = [
    "shocking", "breaking", "exclusive", "unbelievable", "amazing", "incredible",
    "dramatic", "jaw-dropping", "sensational", "revealed", "uncovered", "startling",
    "bizarre", "explosive", "unprecedented", "unexpected", "mind-blowing", "stunning",
    "outrageous", "scandalous", "astonishing", "shocking"
]

densities = {}

for keyword in sensational_words:
    density = keyword_density(text2, keyword)
    print(f"Keyword density of '{keyword}': {density:.2f}%")
    densities[keyword] = density

Keyword density of 'shocking': 0.00%
Keyword density of 'breaking': 0.00%
Keyword density of 'exclusive': 0.00%
Keyword density of 'unbelievable': 0.00%
Keyword density of 'amazing': 0.00%
Keyword density of 'incredible': 0.00%
Keyword density of 'dramatic': 0.00%
Keyword density of 'jaw-dropping': 0.00%
Keyword density of 'sensational': 0.00%
Keyword density of 'revealed': 0.00%
Keyword density of 'uncovered': 0.00%
Keyword density of 'startling': 0.00%
Keyword density of 'bizarre': 0.00%
Keyword density of 'explosive': 0.00%
Keyword density of 'unprecedented': 0.00%
Keyword density of 'unexpected': 0.00%
Keyword density of 'mind-blowing': 0.00%
Keyword density of 'stunning': 0.00%
Keyword density of 'outrageous': 0.00%
Keyword density of 'scandalous': 0.00%
Keyword density of 'astonishing': 0.00%
Keyword density of 'shocking': 0.00%


In [ ]:
headline = "Breaking news. A person is blasting himself."

# Tokenize and POS tag
text2 = " ".join(sensational_words)

tokens = word_tokenize(headline)
tags = pos_tag(tokens)

print(f"Tokens : {tokens}")
print(f"Tags : {tags}")


# Extract adjectives (JJ) and adverbs (RB)
important_words = [word for word, tag in tags if tag in ['JJ', 'RB', 'VBG']]
print(important_words)

print(len(important_words))
print(len(sensational_words))

Tokens : ['Breaking', 'news', '.', 'A', 'person', 'is', 'blasting', 'himself', '.']
Tags : [('Breaking', 'VBG'), ('news', 'NN'), ('.', '.'), ('A', 'DT'), ('person', 'NN'), ('is', 'VBZ'), ('blasting', 'VBG'), ('himself', 'PRP'), ('.', '.')]
['Breaking', 'blasting']
2
22


In [ ]:
from duckduckgo_search import DDGS
from nltk import word_tokenize, pos_tag
import re

def get_sensational_score(word):
    try:
        with DDGS() as ddgs:
            search_query = f'"{word}" "sensational journalism" OR "clickbait"'
            results = list(ddgs.text(search_query, max_results=10))
            return len(results)
    except:
        return 0

def analyze_headline(headline):
    tokens = word_tokenize(headline)
    tags = pos_tag(tokens)
    important_words = [word for word, tag in tags if tag in ['JJ', 'RB', 'VBG']]

    sensational_words = []
    for word in important_words:
        score = get_sensational_score(word)
        if score > 2:
            sensational_words.append(word)

    return {
        'tokens': tokens,
        'tags': tags,
        'important_words': important_words,
        'sensational_words': sensational_words
    }

headline = "Breaking news. A person is blasting himself."
results = analyze_headline(headline)

print(f"Tokens: {results['tokens']}")
print(f"Tags: {results['tags']}")
print(f"Important words: {results['important_words']}")
print(f"Sensational words: {results['sensational_words']}")
print(f"Sensational word count: {len(results['sensational_words'])}")

Tokens: ['Breaking', 'news', '.', 'A', 'person', 'is', 'blasting', 'himself', '.']
Tags: [('Breaking', 'VBG'), ('news', 'NN'), ('.', '.'), ('A', 'DT'), ('person', 'NN'), ('is', 'VBZ'), ('blasting', 'VBG'), ('himself', 'PRP'), ('.', '.')]
Important words: ['Breaking', 'blasting']
Sensational words: ['Breaking']
Sensational word count: 1


In [11]:
new_train_df.to_csv('filename.csv', index=False)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

: 